### Import

In [1]:
# from pyspark import SparkContext #, SparkConf
# from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession

from pyspark.sql import types as T
from pyspark.sql import functions as F

In [2]:
# sc = SparkContext(master='local[*]', appName='Basics')
# ssc = StreamingContext(sc, 60)

In [5]:
# spark = SparkSession.builder.config(sc.getConf()).getOrCreate()
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

# conf = SparkConf().setAppName("Basics").setMaster("local[1]")
# sc = SparkContext(conf=conf)
# spark = SparkSession.builder.config(sc.getConf).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/18 18:03:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Test Data

In [10]:
import pandas as pd

pddf = pd.read_json("data/people-sample.json")
print(pddf.shape)
pddf.head(5)

(4, 2)


,name,age
0,Keeley Bosco,20
1,Rubye Jerde,30
2,Andy Bosco,40
3,John Jerde,None


In [6]:
# rdd = sc.textFile('python/test_support/sql/people.json')
# spark_df = spark.read.json(rdd)

### Test

In [12]:
test_df = spark.read.json('data/people-sample.json')
test_df.show()

+---------------+----+------------+
|_corrupt_record| age|        name|
+---------------+----+------------+
|              [|null|        null|
|           null|  20|Keeley Bosco|
|           null|  30| Rubye Jerde|
|           null|  40|  Andy Bosco|
|           null|None|  John Jerde|
|              ]|null|        null|
+---------------+----+------------+



In [13]:
test_df.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- age: string (nullable = true)
 |-- name: string (nullable = true)



In [15]:
test_df.describe().show()

+-------+---------------+----+-----------+
|summary|_corrupt_record| age|       name|
+-------+---------------+----+-----------+
|  count|              2|   4|          4|
|   mean|           null|30.0|       null|
| stddev|           null|10.0|       null|
|    min|              [|  20| Andy Bosco|
|    max|              ]|None|Rubye Jerde|
+-------+---------------+----+-----------+



### Test 2

In [14]:
data_schema = [T.StructField('age', T.IntegerType(), True),
               T.StructField('name',T.StringType(),True)]
final_struct = T.StructType(fields=data_schema)

df = spark.read.json('data/people-sample.json', schema=final_struct)
df.show()

+----+------------+
| age|        name|
+----+------------+
|null|        null|
|  20|Keeley Bosco|
|  30| Rubye Jerde|
|  40|  Andy Bosco|
|null|  John Jerde|
|null|        null|
+----+------------+



In [22]:
type(df['age'])

pyspark.sql.column.Column

In [25]:
type(df.select('age'))

pyspark.sql.dataframe.DataFrame

In [28]:
type(df.head(2)[1])

pyspark.sql.types.Row

In [34]:
df.select(['age', 'name']).show()

+----+------------+
| age|        name|
+----+------------+
|null|        null|
|  20|Keeley Bosco|
|  30| Rubye Jerde|
|  40|  Andy Bosco|
|null|  John Jerde|
|null|        null|
+----+------------+



In [38]:
df.withColumn('double_age', df['age']*2).show()

+----+------------+----------+
| age|        name|double_age|
+----+------------+----------+
|null|        null|      null|
|  20|Keeley Bosco|        40|
|  30| Rubye Jerde|        60|
|  40|  Andy Bosco|        80|
|null|  John Jerde|      null|
|null|        null|      null|
+----+------------+----------+



In [40]:
df.withColumnRenamed('age', 'my_new_age').show()

+----------+------------+
|my_new_age|        name|
+----------+------------+
|      null|        null|
|        20|Keeley Bosco|
|        30| Rubye Jerde|
|        40|  Andy Bosco|
|      null|  John Jerde|
|      null|        null|
+----------+------------+



In [42]:
df.createOrReplaceTempView('people')

In [44]:
results = spark.sql("SELECT * FROM people")
results.show()

+----+------------+
| age|        name|
+----+------------+
|null|        null|
|  20|Keeley Bosco|
|  30| Rubye Jerde|
|  40|  Andy Bosco|
|null|  John Jerde|
|null|        null|
+----+------------+



In [46]:
new_results = spark.sql("SELECT * FROM people WHERE age=30")
new_results.show()

+---+-----------+
|age|       name|
+---+-----------+
| 30|Rubye Jerde|
+---+-----------+



In [30]:
print(dir(df))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_collect_as_arrow', '_jcols', '_jdf', '_jmap', '_joinAsOf', '_jseq', '_lazy_rdd', '_repr_html_', '_sc', '_schema', '_session', '_sort_cols', '_sql_ctx', '_support_repr_html', '_to_corrected_pandas_type', 'agg', 'alias', 'approxQuantile', 'cache', 'checkpoint', 'coalesce', 'colRegex', 'collect', 'columns', 'corr', 'count', 'cov', 'createGlobalTempView', 'createOrReplaceGlobalTempView', 'createOrReplaceTempView', 'createTempView', 'crossJoin', 'crosstab', 'cube', 'describe', 'distinct', 'drop', 'dropDuplicates', 'drop_duplicates', 'dropna', 'dtypes', 'exceptAll', 'explain', 'fillna', 'filter', 'first', 'foreach', 'fore